Importation des librairies

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.utils import save_image
import numpy as np
import pandas as pd
import random
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset, TensorDataset
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from collections import Counter

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Data Loading

In [ ]:
categories = ["airplane", "backpack", "car", "coffee_cup", "sailboat"]

airplane = np.load('/content/drive/MyDrive/Quickdraw/datasets/full_numpy_bitmap_airplane.npy')
backpack = np.load('/content/drive/MyDrive/Quickdraw/datasets/full_numpy_bitmap_backpack.npy')
car = np.load('/content/drive/MyDrive/Quickdraw/datasets/full_numpy_bitmap_car.npy')
coffee_cup = np.load('/content/drive/MyDrive/Quickdraw/datasets/full_numpy_bitmap_coffee cup.npy')
sailboat = np.load('/content/drive/MyDrive/Quickdraw/datasets/full_numpy_bitmap_sailboat.npy')

image = [airplane[:10000], backpack[:10000],car[:10000], coffee_cup[:10000], sailboat[:10000]]

Preprocessing data

In [ ]:
pic = image[4][7].reshape(28, 28)
plt.imshow(pic, cmap='gray')
plt.show()

In [ ]:
# Création du DataFrame
source = {"Catégorie": [], "image": []}

for cat, arr in zip(categories, image):
    for val in arr:
        source["Catégorie"].append(cat)
        source["image"].append(val)

df = pd.DataFrame(source)

# Affichage du DataFrame
print(df)

Géneration des questions

In [ ]:
# Différents modèles de questions
QUESTION_TEMPLATES = [
    "Peux-tu me dire si c'est un {} ?",
    "Est-ce un {} sur l'image ?",
    "L'image contient-elle un {} ?",
    "Y a-t-il un {} ici ?",
    "Ce dessin représente-t-il un {} ?",
    "On dirait un {}, non ?",
    "S'agit-il d'un {} ?",
]
switcher = {
        "airplane": "avion",
        "backpack": "sac",
        "car": "voiture",
        "coffee_cup": "tasse",
        "sailboat": "bateau",
    }

# Fonction pour générer une question aléatoire avec une catégorie aléatoire
def generate_random_question(true_category):
    category = random.choice(categories)  # Choisir une catégorie au hasard
    if category == true_category:
        category = true_category
    else:
        category = random.choice([category, true_category])
    template = random.choice(QUESTION_TEMPLATES)  # Choisir un modèle de phrase au hasard

    category_fr = switcher.get(category)
    return template.format(category_fr), category # return question and category

# Exemple
true_category = "airplane"
correct = 0
for i in range(100):
    question, category = generate_random_question(true_category)
    if category==switcher.get(true_category):
      correct += 1
print(correct)
print(question)  # Exemple : "L'image contient-elle un avion ?"
print(category)

In [ ]:
# Fonction pour encoder une question
def encode_question(question):
    words = simple_preprocess(question) #convert a document into a list of token.
    vectors = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(w2v_model.vector_size)

# Exemple
question = "Y a-t-il un avion ici ?"
question_vector = encode_question(question)
print("Feature de la question :", question_vector.shape)  # (200,)

In [ ]:
# Fusion des features d’image et de texte
def fuse_features(image_features, question_features):
    return np.concatenate((image_features, question_features))  # (128 + 200 = 328D)

input = fuse_features(image_features, question_vector)
print(input, input.shape)

In [ ]:
# Modification de la boucle d'entraînement du MLP
X_train = []
y_train = []

# Construire le dataset pour le MLP
for _, row in df.iterrows():
    image_vector = row["image"]
    true_category = row["Catégorie"] # Stock la vraie catégorie

    # Extraire les features de l’image
    image_features = extract_image_features(image_vector)

    # Générer une question aléatoire avec une catégorie aléatoire
    question, question_category = generate_random_question(true_category)

    # Encoder la question avec Word2Vec
    question_features = encode_question(question)

    # Fusionner
    joint_representation = fuse_features(image_features, question_features)

    # Définir la réponse (Oui = 1 si c'est la bonne catégorie, sinon Non = 0)
    is_correct_category = (true_category == question_category)  # Comparer avec la vraie catégorie
    label = 1 if is_correct_category else 0

    X_train.append(joint_representation)
    y_train.append(label)

# Convertir en tensors
X = torch.tensor(X_train, dtype=torch.float32).to(device)
y = torch.tensor(y_train, dtype=torch.long).to(device)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"✅ Dataset d'entrainement MLP prêt avec {len(x_train)} exemples.")
print(f"✅ Dataset d'évaluation MLP prêt avec {len(x_test)} exemples.")

In [ ]:
# Compter le nombre de Oui et Non
counter = Counter(y_train.cpu().numpy())
print(f"Avant équilibrage : {counter}")

num_yes = (y_train == 1).sum().item()
num_no = (y_train == 0).sum().item()

# Calculer les poids inverses (moins d'exemples = poids plus fort)
weight_no = num_yes / (num_no + num_yes)
weight_yes = num_no / (num_no + num_yes)

weights = torch.tensor([weight_no, weight_yes], dtype=torch.float32)
criterion = nn.CrossEntropyLoss(weight=weights)

print(f"✅ Poids des classes : Non={weight_no:.2f}, Oui={weight_yes:.2f}")